In [1]:
import sys
import os
import numpy as np
import glob
from tifffile import imread, imwrite
import napari
from pathlib import Path
from skimage.measure import  label

from skimage.morphology import remove_small_objects

In [2]:
Imagedir = '/Users/aimachine/fmi-assignment/Denoised/'
SpotSegmentationMSER = '/Users/aimachine/fmi-assignment/SegmentationResults/SpotSegmentationMSER/'
SpotProbabilityILASTIK = '/Users/aimachine/fmi-assignment/SegmentationResults/SpotProbabilityILASTIK/'
SpotSegmentationILASTIK = '/Users/aimachine/fmi-assignment/SegmentationResults/SpotSegmentationILASTIK/'

In [3]:
#Use the probability image coming out of Ilastik for the (first) spot channel 
#and threshold to remove low probability detections
ProbabilityThreshold = 0.5
#Remove spots below a certain size, to get rid of tiney dots
min_size = 20

In [4]:
Raw_path = os.path.join(Imagedir, '*tif')
Path(SpotSegmentationILASTIK).mkdir(exist_ok = True)
SegSpot_path = os.path.join(SpotSegmentationMSER, '*tif')
SegSpotIlastik_path = os.path.join(SpotProbabilityILASTIK, '*tif')
SegSpotSegIlastik_path = os.path.join(SpotSegmentationILASTIK, '*tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort

filesSegSpot = glob.glob(SegSpot_path)
filesSegSpot.sort
filesSegSpotIlastik = glob.glob(SegSpotIlastik_path)
filesSegSpotIlastik.sort
filesSegSpotSegIlastik = glob.glob(SegSpotSegIlastik_path)
filesSegSpotSegIlastik.sort

<function list.sort(*, key=None, reverse=False)>

# Convert Ilastik Probability map of two channel to label images

In [5]:
for fname in filesSegSpotIlastik:
    Name = os.path.basename(os.path.splitext(fname)[0])
    TwoChannel = imread(fname)
    SpotChannel = TwoChannel[:,0,:,:]
    Binary = SpotChannel > ProbabilityThreshold
    Binary = remove_small_objects(Binary, min_size = min_size)
    Integer = label(Binary)
    imwrite(SpotSegmentationILASTIK + Name + '.tif', Integer.astype('uint16'))    

# Compare the MSER and ILASTIK segmentations

In [7]:
for fname in filesRaw:
    for fourthname in filesSegSpot: 
               for fifthname in filesSegSpotSegIlastik:
                     FirstName = os.path.basename(os.path.splitext(fname)[0])
                     SegSpotName = os.path.basename(os.path.splitext(fourthname)[0] )    
                     SegSpotIlastikName = os.path.basename(os.path.splitext(fifthname)[0] )
                     if FirstName == SegSpotName == SegSpotIlastikName :
                        image = imread(fname)
                        SegSpotimage = imread(fourthname)                        
                        SegSpotIlastikimage = imread(fifthname)
                        Name = (os.path.basename(os.path.splitext(fname)[0]))
                        with napari.gui_qt():


                            viewer = napari.view_image(image, name='Raw')
                            viewer.add_labels(SegSpotimage, name= 'SpotsMSER')
                            viewer.add_labels(SegSpotIlastikimage, name= 'SpotsIlastik')
                            